#  Assignment 5

## Notebook made by  

|** Name** | **Student id** | **email**|
|:- |:-|:-|
|. | | |
|  | |. |

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. 

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='link to your selfie'/>

### Note
* **Assignments without the selfies or completely filled in information will not be graded and receive 0 points.**


We maken de toetsassignment af. We bekijken dus weer het artikel <http://language.media.mit.edu/paper>. In de toets hebben we de netwerkdata gebaseerd op boekvertalingen gemaakt.
In dit huiswerk gaan we verder.

 

In [2]:
%matplotlib inline
import pandas as pd
import re
from lxml import etree 
from bz2file import BZ2File
import codecs
import nltk
from collections import defaultdict
from  itertools import combinations  # ideal for creating all possible pairs that out can make out of a set
from __future__ import division
from math import sqrt 

### Deel  Wikipedia  

 

1. Maak   van de Wikipedia _raw_ dataset de netwerk  datasets die op <http://language.media.mit.edu/data> beschreven staan. 
2. We bekijken in hoeverre onze resultaten afwijken van die in de geleverde bewerkte datasets.
3. We leggen eventuele verschillen uit.

#### In Stapjes

We doen de opdracht in een aantal stapjes.

#### Let op, je notebook moet vanzelf draaien. Alle data moet opgehaald worden.

# Download 

1. Download de ruwe dataset als zip file, ontzip hem en sla hem in de directory op waarin dit notebook ook staat.
    * Dit mag je met de hand doen. De code **moet niet runbaar** in je notebook staan.
    * Maar zorg dat de rest van de code er vanuit gaat dat `wikipedia_userlang_iso639-3.tsv` in dezelfde folder staat als dit notebook
2. Lees de 2 bewerkte datasets over Wikipedia _Wikipedia Stats by Language_ en _Edges_ direct in van de urls in  pandas Dataframes, die je respectivelijk `dataset_stats_wikipedia` en `wikipedia_edges` noemt.
    1. Maak de Taal code de index in het eerste frame, en hou alleen de Kolommen `['Editors', 'Edits','Multilinguals','Edits by mulitlinguals']`
    2. Sorteer het frame op de index
    2. Verwijder de kolommen `SourceLanguageName	TargetLanguageName	SourcePopulation	TargetPopulations` uit het tweede frame
    3. Sorteer het frame op `SourceLanguageName`
    3. Laat steeds met `head()` zien dat het werkt.
    
### Je antwoord bestaat uit 2 cellen

In [15]:
#download the data
#done it by hand in the browser


# jouw code hier 
dataset_stats_wikipedia = pd.read_csv("http://language.media.mit.edu/data/public/dataset_stats_wikipedia.tsv", sep = '\t')
dataset_stats_wikipedia = dataset_stats_wikipedia.set_index('Code')
dataset_stats_wikipedia = dataset_stats_wikipedia[['Editors', 'Edits','Multilinguals','Edits by mulitlinguals']].sort_index()
dataset_stats_wikipedia.head()


,Editors,Edits,Multilinguals,Edits by mulitlinguals
Code,,,,
aar,4,46,3,9
abk,15,2807,7,1494
afr,511,103652,352,95988
aka,31,327,24,240
amh,68,4596,45,4039


In [22]:
# jouw code hier 

wikipedia_edges = pd.read_csv('http://language.media.mit.edu/data/public/wikipedia_edges.tsv', sep = '\t')
wikipedia_edges = wikipedia_edges.drop(['SourceLanguageName', 'TargetLanguageName', 'SourcePopulation','TargetPopulations'], axis = 1).set_index('SourceLanguageCode')
wikipedia_edges.head()

,TargetLanguageCode,Coocurrences,PhiCorrelation,Tstatistic
SourceLanguageCode,,,,
afr,eng,271,0.013648,17.207148
afr,nld,78,0.015377,3.332197
amh,eng,28,0.004344,5.476711
ang,eng,51,0.006363,8.021067
ara,deu,446,0.007497,3.550224


# Haal gegevens uit de ruwe data, deel 1

* De file `'wikipedia_userlang_iso639-3.tsv'` lijkt een spreadsheet maar hij heeft niet op elke regel evenveel kolommen.
* Processing met pandas is dus ook lastig.
* Als alternatief gaan we file zelf regel voor regel  inlezen, en de gewenste gegevens opslaan.
* Ons einddoel is om een dataframe te maken dat er precies zo uitziet als `dataset_stats_wikipedia`
* We doen dat in 2 stappen:
    1. We maken een aantal dicts aan met de gewenste gegevens
    2. Van elke dict maken we een dataframe met `pd.DataFrame.from_dict`
    3. Die plakken we allemaal aan elkaar met `join` (neem een inner join)
    4. Noem het eindresultaat `stats`, sorteer het ook op de index en laat het begin zien met `head`.
    5. Check natuurlijk zelf of je op de goede weg zit door je resultaat te vergelijken met `dataset_stats_wikipedia`
* Hieronder staat wat code om je op weg te helpen
* **Lees het artikel en de supplementary material goed om net als zij de ruwe data te filteren.**


# Haal gegevens uit de ruwe data, deel 2

* In dezelfde code waarin je de vorige vraag hebt gemaakt, maak je ook een dict `pairs` aan. Hier zijn de sleutels paren van taalcodes, **waarbij de eerste altijd alfabetisch eerder is dan de tweede**, en hun waarde het aantal cooccurences.
* Maak van dit dict een Dataframe `edges`, waarbij `edges.columns=['SourceLanguageCode','TargetLanguageCode','Coocurrencesmy']`
* Sorteer ook hier op `'SourceLanguageCode'`
* **Lees het artikel en de supplementary material goed om net als zij de ruwe data te filteren.**

In [15]:
# Your code 
with open('wikipedia_userlang_iso639-3.tsv') as f:
    c=0
    editors=defaultdict(int)
    edits=defaultdict(int)
    multilingual=defaultdict(int)
    Edits_by_multilinguals=defaultdict(int)
    pairs= defaultdict(int)
    pairs = {}
    c=0
    z=defaultdict(int)
    for l in f:
        
        a = l.replace('\n',"").split('\t')
        #maximal 5 languages
        if len(a) < 6:
            
            for i in a[1:]:
                
                x = sum([int(i.split(',')[-1])])
                
                
                #at least two edits and more than 4 per language
                if int(i.split(',')[1]) >=2 and x > 4:
                    
                    
                    
                                 #original dicts
                    editors[i.split(',')[0]] +=1
                    edits[i.split(',')[0]] += int(i.split(',')[1])
                    if len(a) >= 3:
                 
                        multilingual[i.split(',')[0]] +=1
                        Edits_by_multilinguals[i.split(',')[0]] += int(i.split(',')[1])
               
                        
                
                        all_languages = sorted([i.split(',')[0] for i in a[1:]])
                    
                        
                        combs = [i for i in combinations(all_languages, 2)]
                        for pair in combs:
                            if pair in pairs:
                                pairs[pair] += 1
                            else: 
                                pairs[pair] = 1 

                    
                 
                    
                
                    
                
                    
                    
                        
               
        
    

  


In [13]:
len(pairs)

0

In [16]:
#Deel 1
# jouw code hier 
editorframe = pd.DataFrame.from_dict(dict(editors), orient='index')
editorframe.rename(columns={0:'Editors'}, inplace=True)

editframe = pd.DataFrame.from_dict(edits, orient='index')
editframe.rename(columns={0:'Edits'}, inplace=True)

multiframe = pd.DataFrame.from_dict(multilingual, orient='index')
multiframe.rename(columns={0:'Multilinguals'}, inplace=True)

edits_by_multiframe = pd.DataFrame.from_dict(Edits_by_multilinguals, orient='index')
edits_by_multiframe.rename(columns={0:'Edits by mulitlinguals'}, inplace=True)

stats = pd.DataFrame.join(editorframe, [editframe, multiframe, edits_by_multiframe], how='inner')
stats.index.name= "Code"
stats.sort_index().head()


,Editors,Edits,Multilinguals,Edits by mulitlinguals
Code,,,,
aar,1,37,1,37
abk,12,2798,6,1523
afr,348,57809,218,51858
aka,15,258,9,191
amh,47,4513,24,3956


In [17]:
#Deel2
tuples = pairs.keys()

# Lists are ordered so the unzipped order matches
source = zip(*pairs.keys())[0]
target = zip(*pairs.keys())[1]

coocurrences = [pairs[tup] for tup in tuples]

edges = pd.DataFrame(zip(source, target, coocurrences), 
                     columns=['SourceLanguageCode','TargetLanguageCode',
                              'Coocurrencesmy']).sort_values(by='SourceLanguageCode')

edges.head(10) # Klopt niet


,SourceLanguageCode,TargetLanguageCode,Coocurrencesmy
1969,aar,xho,2
1126,aar,swe,1
2553,aar,deu,2
897,aar,ron,1
1305,aar,ita,3
518,aar,arg,1
3081,aar,bis,2
4223,aar,eng,2
3713,abk,jpn,2
4065,abk,tur,2


# Vergelijk 

We gaan nu onze dataframes vergelijken met die we hebben opgehaald.

We zijn natuurlijk erg benieuwd of er verschillen zijn, en vooral ook groot die dan zijn, als ze er zijn.

1. Join `dataset_stats_wikipedia` en `stats` in een nieuwe dataframe `samen`. 
    * laat met `samen.describe()` zien hoe groot of klein de verschillen zijn
2. Maak een functie `fout_percentage` waar je twee kolommen en een dataframe aan geeft en die _het verschil tussen die twee kolommen bepaalt, en omzet naar een "foutpercentage"_.
3. Bereken dat percentage voor de vier kolommen `['Editors', 'Edits','Multilinguals','Edits by mulitlinguals']` en print het netjes uit.
4. Leg in een paar zinnen uit waarom jouw maat voor "foutpercentage" echt ergens op slaat, en het een goede indicator geeft. 
5. Doe tenslotte hetzelfde voor `edges`. 
    * Hier doe je natuurlijk een `merge`
    * Bekijk hier eens het verschil tussen een outer en een inner join.
    * Leg uit wat er hier aan de hand is.
    * Kan je het verschil verklaren?


In [230]:
# vergelijk stats met dataset_stats_wikipedia

samen=

def fout_percentage(col1,col2,df):
     # jouw code hier 

 

for col in stats.columns:
    print "%s\t%s\t%s" %(col, fout_percentage(col, col+'my', samen)

Editors	0.518	4689
Edits	0.014	17285
Multilinguals	0.349	503
Edits by mulitlinguals	0.003	1006


In [231]:
samen.describe()

,Editors,Edits,Multilinguals,Edits by mulitlinguals,Editorsmy,Editsmy,Multilingualsmy,Edits by mulitlingualsmy
count,238.000000,2.380000e+02,238.000000,238.000000,238.000000,2.380000e+02,238.000000,238.000000
mean,12243.554622,1.608757e+06,2648.226891,729135.878151,12307.088235,1.608984e+06,2657.487395,729154.399160
std,105148.317861,1.322120e+07,15606.769464,5342618.270859,105478.073646,1.322242e+07,15647.386691,5342696.119103
min,1.000000,2.000000e+00,1.000000,2.000000,1.000000,2.000000e+00,1.000000,2.000000
25%,20.250000,1.410500e+03,13.000000,562.500000,21.000000,1.413500e+03,13.000000,562.500000
50%,61.500000,8.092500e+03,39.000000,5325.500000,63.500000,8.094500e+03,39.000000,5325.500000
75%,458.250000,1.099002e+05,238.500000,64802.000000,463.250000,1.099082e+05,240.000000,64802.000000
max,1589250.000000,1.983610e+08,218320.000000,78341211.000000,1593939.000000,1.983783e+08,218823.000000,78342217.000000


In [ ]:
# vergelijk edges met "inner join"
 

In [ ]:
# vergelijk edges met "outer join"

edges2=pd.merge(edges,wikipedia_edges, how='outer', on=['SourceLanguageCode','TargetLanguageCode'])
print fout_percentage('Coocurrencesmy','Coocurrences', edges2)
edges2.describe()

# Compute our own PhiCorrelation and T-statistic

Maak een functie `PhiCorrelation(i,j)` die gegeven 2 talen, de $\phi$ correlatie en de $t$-statistic uitrekent volgens vergelijkingen [1] en [2] uit het artikel.

* Let vooral op hoe je $N$ bepaalt. En beschrijf goed waarom je precies die keuze hebt genomen. Je uitkomsten hangen daar sterk vanaf.

In [213]:
# Compute our own PhiCorrelation and T-statistic


def PhiCorrelation(i,j):
     #Your code here
    return Mij,Mi, Mj,  N, PhiCorrelation, Tstatistic

# test
PhiCorrelation('afr','nld')

(79, 531, 26706, 2929087, 0.019785094309992902, 3.2337861998200852)

# Reflectie
Je hebt getracht een deel van het artikel te reproduceren. Ook hebben we gedacht over het artikel en de mate van herhaalbaarheid van de vermelde gegevens. 

Geef nu in een half A4 je eigen mening. Behandel deze twee punten:

1. Wat was al goed gedaan?
2. Wat kon beter en hoe dan?